In [17]:
import pandas as pd
import numpy as np
import json

In [18]:
ENTRY_DATE = "240224"
USER = 'quan_' 

In [19]:
raw_data = json.load(open(f'/workspace/nlplab/quannd/scada-full-stack/process-data/data-source/{USER}entry_date_{ENTRY_DATE}.json', 'r'))
raw_data

[{'text': '<i> Di chuyển hình chữ nhật màu đen , kích thước 50x60 pixel , từ vị trí hiện tại đến vị trí mới tại ( 200 , 300 ) để góc dưới trùng với vị trí đích .',
  'id': 1356,
  'label': [{'start': 14,
    'end': 27,
    'text': 'hình chữ nhật',
    'labels': ['object|shape']},
   {'start': 32, 'end': 35, 'text': 'đen', 'labels': ['object|color']},
   {'start': 38, 'end': 48, 'text': 'kích thước', 'labels': ['object|length']},
   {'start': 66, 'end': 72, 'text': 'vị trí', 'labels': ['aspect']},
   {'start': 86, 'end': 92, 'text': 'vị trí', 'labels': ['aspect']},
   {'start': 103,
    'end': 106,
    'text': '200',
    'labels': ['position|destination_x']},
   {'start': 109,
    'end': 112,
    'text': '300',
    'labels': ['position|destination_y']},
   {'start': 118, 'end': 126, 'text': 'góc dưới', 'labels': ['aspect']},
   {'start': 137, 'end': 143, 'text': 'vị trí', 'labels': ['aspect']},
   {'start': 0, 'end': 3, 'text': '<i>', 'labels': ['intent|change_bottom']},
   {'start': 49

In [20]:
print(raw_data[10]['text'])
print(raw_data[10]['label'])

<i> Hãy di chuyển hình tròn màu xanh lá cây , bán kính 25 pixel , từ vị trí góc dưới là ( 200 , 100 ) đến vị trí mới tại ( 150 , 250 ) - sự điều chỉnh từ góc dưới một cách tinh tế .
[{'start': 0, 'end': 3, 'text': '<i>', 'labels': ['intent|change_bottom']}, {'start': 18, 'end': 27, 'text': 'hình tròn', 'labels': ['object|shape']}, {'start': 32, 'end': 43, 'text': 'xanh lá cây', 'labels': ['object|color']}, {'start': 46, 'end': 54, 'text': 'bán kính', 'labels': ['aspect']}, {'start': 55, 'end': 63, 'text': '25 pixel', 'labels': ['object|width']}, {'start': 69, 'end': 75, 'text': 'vị trí', 'labels': ['aspect']}, {'start': 76, 'end': 84, 'text': 'góc dưới', 'labels': ['aspect']}, {'start': 90, 'end': 93, 'text': '200', 'labels': ['position|source_x']}, {'start': 96, 'end': 99, 'text': '100', 'labels': ['position|source_y']}, {'start': 106, 'end': 112, 'text': 'vị trí', 'labels': ['aspect']}, {'start': 123, 'end': 126, 'text': '150', 'labels': ['position|destination_x']}, {'start': 129, 'e

In [21]:
ENTITY_LIST = {}

In [22]:
def convert_sample(sample):
    sent = sample['text'][4:]
    # if 'label' in sample:
    #     print(sample)
    #     return 
    label = sample['label']

    label_tuple = []
    for item in label:
        if '<i>' in item['text']:
            assert len(item['labels']) == 1, f'Got overlapse labels for intent!'
            intent = item['labels'][0][7:]
        else:
            label_tuple.append(item)
    label_tuple.sort(key=lambda x : x['start'])
    last_e_index = 0
    cv_sent = ''
    if len(label_tuple) == 0:
        return intent, sent
    
    for item in label_tuple:
        # print(item)
        assert len(item['labels']) == 1, f'Got overlapse labels for entity!'
        s_index = item['start']
        e_index = item['end']
        text = item['text']
        # print(item['labels'][0])
        try:
            entity, role = item['labels'][0].split('|')
            if entity in ENTITY_LIST.keys():
                if role not in ENTITY_LIST[entity]: ENTITY_LIST[entity].append(role)
            else:
                ENTITY_LIST[entity] = [role]
        except Exception:
            entity, role = item['labels'][0], None
            if entity in ENTITY_LIST.keys():
                ENTITY_LIST[entity] = []

        if entity == 'intent': 
            print(sample)

        cv_sent += sent[last_e_index : s_index-4]

        cv_sent += '[' + text + ']{"entity": "' + entity + '"'
        cv_sent += ', "role": "' + role + '"}' if role is not None else '}'

        last_e_index = e_index-4

    return intent, cv_sent

In [23]:
data_dict = {}
for sample in raw_data:
    print(sample)
    intent, cv_sent = convert_sample(sample)
    if intent not in data_dict.keys():
        data_dict[intent] = []
    else:
        data_dict[intent].append(cv_sent)
    
data_dict

{'text': '<i> Di chuyển hình chữ nhật màu đen , kích thước 50x60 pixel , từ vị trí hiện tại đến vị trí mới tại ( 200 , 300 ) để góc dưới trùng với vị trí đích .', 'id': 1356, 'label': [{'start': 14, 'end': 27, 'text': 'hình chữ nhật', 'labels': ['object|shape']}, {'start': 32, 'end': 35, 'text': 'đen', 'labels': ['object|color']}, {'start': 38, 'end': 48, 'text': 'kích thước', 'labels': ['object|length']}, {'start': 66, 'end': 72, 'text': 'vị trí', 'labels': ['aspect']}, {'start': 86, 'end': 92, 'text': 'vị trí', 'labels': ['aspect']}, {'start': 103, 'end': 106, 'text': '200', 'labels': ['position|destination_x']}, {'start': 109, 'end': 112, 'text': '300', 'labels': ['position|destination_y']}, {'start': 118, 'end': 126, 'text': 'góc dưới', 'labels': ['aspect']}, {'start': 137, 'end': 143, 'text': 'vị trí', 'labels': ['aspect']}, {'start': 0, 'end': 3, 'text': '<i>', 'labels': ['intent|change_bottom']}, {'start': 49, 'end': 51, 'text': '50', 'labels': ['object|length']}, {'start': 52, 

{'change_bottom': ['Hãy di chuyển [hình tròn]{"entity": "object", "role": "shape"} [đỏ]{"entity": "object", "role": "color"} , [bán kính]{"entity": "aspect"} [40 pixel]{"entity": "object", "role": "width"} , từ [vị trí]{"entity": "aspect"} [góc dưới]{"entity": "aspect"} là ( [150]{"entity": "position", "role": "source_x"} , [200]{"entity": "position", "role": "source_y"} ) đến [vị trí]{"entity": "aspect"} mới tại ( [100]{"entity": "position", "role": "destination_x"} , [150]{"entity": "position", "role": "destination_y"} ) - sự điều chỉnh tinh tế từ [góc dưới]{"entity": "aspect"}',
  'Thực hiện hành động di chuyển cho [hình vuông]{"entity": "object", "role": "shape"} màu [vàng]{"entity": "object", "role": "color"} , [kích thước]{"entity": "aspect"} [25 pixel]{"entity": "object", "role": "width"} , đến [vị trí]{"entity": "aspect"} [góc dưới]{"entity": "aspect"} là ( [400]{"entity": "position", "role": "destination_x"} , [100]{"entity": "position", "role": "destination_y"}',
  'Dịch chuy

In [24]:
ENTITY_LIST

{'object': ['shape', 'color', 'length', 'width', 'height'],
 'position': ['destination_x',
  'destination_y',
  'source_x',
  'source_y',
  'source'],
 'value': ['color', 'angle']}

In [25]:
data_dict['ENTITY_LIST'] = ENTITY_LIST

In [26]:
data_dict

{'change_bottom': ['Hãy di chuyển [hình tròn]{"entity": "object", "role": "shape"} [đỏ]{"entity": "object", "role": "color"} , [bán kính]{"entity": "aspect"} [40 pixel]{"entity": "object", "role": "width"} , từ [vị trí]{"entity": "aspect"} [góc dưới]{"entity": "aspect"} là ( [150]{"entity": "position", "role": "source_x"} , [200]{"entity": "position", "role": "source_y"} ) đến [vị trí]{"entity": "aspect"} mới tại ( [100]{"entity": "position", "role": "destination_x"} , [150]{"entity": "position", "role": "destination_y"} ) - sự điều chỉnh tinh tế từ [góc dưới]{"entity": "aspect"}',
  'Thực hiện hành động di chuyển cho [hình vuông]{"entity": "object", "role": "shape"} màu [vàng]{"entity": "object", "role": "color"} , [kích thước]{"entity": "aspect"} [25 pixel]{"entity": "object", "role": "width"} , đến [vị trí]{"entity": "aspect"} [góc dưới]{"entity": "aspect"} là ( [400]{"entity": "position", "role": "destination_x"} , [100]{"entity": "position", "role": "destination_y"}',
  'Dịch chuy

In [27]:
print(json.dumps(data_dict, ensure_ascii=False, indent=4))

{
    "change_bottom": [
        "Hãy di chuyển [hình tròn]{\"entity\": \"object\", \"role\": \"shape\"} [đỏ]{\"entity\": \"object\", \"role\": \"color\"} , [bán kính]{\"entity\": \"aspect\"} [40 pixel]{\"entity\": \"object\", \"role\": \"width\"} , từ [vị trí]{\"entity\": \"aspect\"} [góc dưới]{\"entity\": \"aspect\"} là ( [150]{\"entity\": \"position\", \"role\": \"source_x\"} , [200]{\"entity\": \"position\", \"role\": \"source_y\"} ) đến [vị trí]{\"entity\": \"aspect\"} mới tại ( [100]{\"entity\": \"position\", \"role\": \"destination_x\"} , [150]{\"entity\": \"position\", \"role\": \"destination_y\"} ) - sự điều chỉnh tinh tế từ [góc dưới]{\"entity\": \"aspect\"}",
        "Thực hiện hành động di chuyển cho [hình vuông]{\"entity\": \"object\", \"role\": \"shape\"} màu [vàng]{\"entity\": \"object\", \"role\": \"color\"} , [kích thước]{\"entity\": \"aspect\"} [25 pixel]{\"entity\": \"object\", \"role\": \"width\"} , đến [vị trí]{\"entity\": \"aspect\"} [góc dưới]{\"entity\": \"aspec

In [28]:
with open(f'./data-converted/{USER}entry_date_{ENTRY_DATE}.json', 'w', encoding='utf8') as json_file:
    json.dump(data_dict, json_file, ensure_ascii=False, indent=4)

In [29]:
data_dict.keys()

dict_keys(['change_bottom', 'color_background', 'ENTITY_LIST'])

In [30]:
data_dict.values()

dict_values([['Hãy di chuyển [hình tròn]{"entity": "object", "role": "shape"} [đỏ]{"entity": "object", "role": "color"} , [bán kính]{"entity": "aspect"} [40 pixel]{"entity": "object", "role": "width"} , từ [vị trí]{"entity": "aspect"} [góc dưới]{"entity": "aspect"} là ( [150]{"entity": "position", "role": "source_x"} , [200]{"entity": "position", "role": "source_y"} ) đến [vị trí]{"entity": "aspect"} mới tại ( [100]{"entity": "position", "role": "destination_x"} , [150]{"entity": "position", "role": "destination_y"} ) - sự điều chỉnh tinh tế từ [góc dưới]{"entity": "aspect"}', 'Thực hiện hành động di chuyển cho [hình vuông]{"entity": "object", "role": "shape"} màu [vàng]{"entity": "object", "role": "color"} , [kích thước]{"entity": "aspect"} [25 pixel]{"entity": "object", "role": "width"} , đến [vị trí]{"entity": "aspect"} [góc dưới]{"entity": "aspect"} là ( [400]{"entity": "position", "role": "destination_x"} , [100]{"entity": "position", "role": "destination_y"}', 'Dịch chuyển [hình 

In [43]:
import yaml

with open(f"/workspace/nlplab/quannd/scada-full-stack/process-data/{USER}nlu_entry_date_{ENTRY_DATE}.yml", 'w', encoding='utf8') as f_out:
    doc = yaml.dump(data_dict, f_out, allow_unicode=True, width=100000)